<a href="https://colab.research.google.com/github/ArshadP443/Rotating-Screw-CNN/blob/main/Copy_of_final_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import re
from time import time

import numpy as np
import pandas as pd

import torch

import torch.nn as nn
from torch.nn import functional as F

from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
class FramesDataset(Dataset):
  def __init__(self, data_pd):
    self.data_pd = data_pd

  def __len__(self):
    return len(self.data_pd)

  def __getitem__(self, index):
    return self.data_pd.iloc[index]['Tensor'], self.data_pd.iloc[index]['Label']

In [ ]:
os.chdir('/content/drive/MyDrive/Object Detection/TrainTensors')
data_df = torch.load('AugementTrain.pt')

In [ ]:
for x in range(len(data_df['Label'])):
  if data_df['Label'].iloc[x] == 'False':
    data_df['Label'].iloc[x] = 0
  else:
    data_df['Label'].iloc[x] = 1

In [ ]:
data_df["Label"].value_counts()

,count
Label,
1,17097
0,13090


In [ ]:
os.chdir('/content/drive/MyDrive/Object Detection/TestTensors')
data_df_test = torch.load('AugementTest.pt')

In [ ]:
for x in range(len(data_df_test['Label'])):
  if data_df_test['Label'].iloc[x] == '0' or 0:
    data_df_test['Label'].iloc[x] = 0
  elif data_df_test['Label'].iloc[x] == '1' or 1:
    data_df_test['Label'].iloc[x] = 1

In [ ]:
data_df_test["Label"].value_counts()

,count
Label,
1,8487
0,6058


In [ ]:
train_dataset = FramesDataset(data_df)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)

In [ ]:
test_dataset = FramesDataset(data_df_test)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 4, 5, 2)
        self.conv2 = nn.Conv2d(4, 16, 5, 2)
        self.fc1 = nn.Linear(576, 550)
        self.fc2 = nn.Linear(550, 540)
        self.fc3 = nn.Linear(540, 470)
        self.fc4 = nn.Linear(470, 300)
        self.fc5 = nn.Linear(300, 220)
        self.fc6 = nn.Linear(220, 130)
        self.fc7 = nn.Linear(130, 70)
        self.fc8 = nn.Linear(70, 40)
        self.fc9 = nn.Linear(40, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 3, 3)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 3, 3)
        x = x.flatten(start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        x = self.fc6(x)
        x = self.fc7(x)
        x = self.fc8(x)
        x = self.fc9(x)
        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = CNN().to(device)

cuda


In [ ]:
# Hyperparameters
epochs = 3
learning_rate = .001
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

param_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {param_count:,} trainable parameters')

The model has 1,119,181 trainable parameters


In [ ]:
model = torch.compile(model, mode="reduce-overhead")
torch.set_float32_matmul_precision("highest")
torch.backends.cudnn.benchmark = True

In [ ]:
for batch_idx, (data, targets) in enumerate(train_dataloader):
    outputs = model(data.to(device)).squeeze(1)
    print(outputs.size())
    loss = criterion(outputs, targets.to(device).float())
    print(loss.item())
    break

torch.Size([64])
0.6863588690757751


In [ ]:
times = []
losses = []

for epoch in range(epochs):
    for batch_idx, (data, targets) in enumerate(train_dataloader):

        start = time()
        outputs = model(data.to(device)).squeeze(1)
        loss = criterion(outputs, targets.to(device).float())

        optimizer.zero_grad()

        loss.backward()
        optimizer.step()


        losses.append(loss.item())
        times.append(time() - start)

        msg =  f'Epoch: {epoch+1:02}/{epochs:02} | '
        msg += f'Batch: {batch_idx+1:05}/{len(train_dataloader):05} | '
        msg += f'Loss: {losses[-1]:0.4} | '
        msg += f'Avg Loss: {sum(losses)/len(losses):0.4} | '
        msg += f'First Prediction: {outputs[0]} | '
        msg += f'First Label: {targets[0]} | '
        msg += f'Time: {times[-1]:0.4} | '
        avg_time = sum(times)/len(times)
        msg += f'Avg Time: {avg_time:0.4} | '
        msg += f'Time Left: {avg_time * (len(train_dataloader) - (batch_idx + 1)):0.4}'
        print(msg, flush=True)
        if (batch_idx%10 == 0):
            torch.cuda.empty_cache()

In [ ]:
predictions = []
labels = []

with torch.no_grad():

    model.eval()

    for batch_idx, (data, targets) in enumerate(test_dataloader):

        outputs = model(data.to(device))
        # print(outputs)
        predicted, _ = torch.max(outputs, 1)
        # print(predicted)
        # print(predicted[0])
        # break
        for i in range(len(predicted)):
          if predicted[i].item() < 0:
            predictions.append(0)
          else:
            predictions.append(1)
        # predictions.extend(predicted.cpu().numpy())
        labels.extend(targets.cpu().numpy())

In [ ]:
print(accuracy_score(labels, predictions))
print(confusion_matrix(labels, predictions))
print(classification_report(labels, predictions))

0.9947060845651426
[[5982   76]
 [   1 8486]]
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      6058
           1       0.99      1.00      1.00      8487

    accuracy                           0.99     14545
   macro avg       1.00      0.99      0.99     14545
weighted avg       0.99      0.99      0.99     14545



In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Object Detection/bestmodels/AugmentCNN.pt')